# Stage 13: Productization — Fraud Detection API Demo

This notebook demonstrates how to prepare the fraud detection project for reuse, handoff, and deployment. It includes examples of loading/training the model, launching the Flask API, and testing endpoints.


## 1. Setup and Load/Train Model


In [1]:
from pathlib import Path
import os, sys, time, json, subprocess, signal
import requests
import joblib

# Find project root (assuming notebook is in project/notebooks/)
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print('PROJECT_ROOT:', PROJECT_ROOT)
print('Current working directory:', Path.cwd())


PROJECT_ROOT: /Users/Kushal/FRE/bootcamp_kushal_mamillapalli/project
Current working directory: /Users/Kushal/FRE/bootcamp_kushal_mamillapalli/project/notebooks


In [2]:
# Import all functions from model_service.py (exact imports from app.py)
from src.model_service import (
    FEATURE_COLUMNS,
    find_default_dataset,
    load_dataset,
    load_model,
    predict_label,
    predict_proba,
    train_and_save,
    get_project_root,
    get_model_dir,
    get_default_model_path,
)

# Use the exact path functions from model_service.py
MODEL_PATH = get_default_model_path()  # Uses model_service function
APP_PATH = PROJECT_ROOT / 'app.py'

print('MODEL_PATH (from model_service):', MODEL_PATH)
print('MODEL_PATH exists:', MODEL_PATH.exists())
print('APP_PATH exists:', APP_PATH.exists())
print('Project root (from model_service):', get_project_root())

# Use the exact train_and_save function from model_service.py
if not MODEL_PATH.exists():
    print('No model found. Training and saving using model_service.train_and_save()...')
    info = train_and_save()  # Uses default dataset finding from model_service
    print('Training completed:', info)
else:
    print('Model already present at', MODEL_PATH)


MODEL_PATH (from model_service): /Users/Kushal/FRE/bootcamp_kushal_mamillapalli/project/model/model.pkl
MODEL_PATH exists: True
APP_PATH exists: True
Project root (from model_service): /Users/Kushal/FRE/bootcamp_kushal_mamillapalli/project
Model already present at /Users/Kushal/FRE/bootcamp_kushal_mamillapalli/project/model/model.pkl


## 2. Test Model and Define Prediction Function


In [ ]:
# Load the model using model_service.py functions
model, feats = load_model()  # Uses default path from model_service
print('Loaded model; feature count:', len(feats))
print('Required features:', feats[:5], '...')
print('Feature columns from model_service:', FEATURE_COLUMNS[:5], '...')

def predict_fraud(amount=100.0, time=0.0, threshold=0.5):
    """
    Predict fraud using model_service functions.
    Uses the exact predict_proba and predict_label functions from model_service.py
    """
    # Create a row using the exact FEATURE_COLUMNS from model_service
    row = {c: 0.0 for c in FEATURE_COLUMNS}
    row['Amount'] = float(amount)
    row['Time'] = float(time)
    
    # Use the exact functions from model_service.py
    probs = predict_proba(model, [row], feats)
    labels = predict_label(model, [row], feats, threshold=threshold)
    
    return {
        'probability': probs[0], 
        'prediction': labels[0], 
        'threshold': threshold,
        'features_used': {'Amount': amount, 'Time': time}
    }


# Test the function using model_service functions
result = predict_fraud(amount=123.45, time=10)
print('Fraud prediction test:', result)

# Test the train_and_save function from model_service
print('\nTesting train_and_save function...')
try:
    dataset_path = find_default_dataset()
    print('Found dataset:', dataset_path)
    
    # This will train and save if model doesn't exist, or just return info if it does
    training_result = train_and_save()
    print('Training result:', training_result)
except Exception as e:
    print('Training test failed:', e)


Loaded model; feature count: 30
Required features: ['Time', 'V1', 'V2', 'V3', 'V4'] ...
Feature columns from model_service: ['Time', 'V1', 'V2', 'V3', 'V4'] ...
Fraud prediction test: {'probability': 1.0, 'prediction': 1, 'threshold': 0.5, 'features_used': {'Amount': 123.45, 'Time': 10}}

Testing train_and_save function...
Found dataset: /Users/Kushal/FRE/bootcamp_kushal_mamillapalli/project/data/processed/fraud_kaggle_creditcard_source-kaggle_dataset-creditcardfraud_20250820-213245_cleaned.csv


/Users/Kushal/miniconda3/envs/fe-course/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/Kushal/miniconda3/envs/fe-course/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Training result: {'model_path': '/Users/Kushal/FRE/bootcamp_kushal_mamillapalli/project/model/model.pkl', 'pr_auc': '0.7190', 'f1': '0.1144', 'precision': '0.0610', 'recall': '0.9184'}


## 3. Launch Flask API and Test Endpoints


In [4]:
# Check if Flask is already running and kill if necessary
try:
    test_resp = requests.get('http://127.0.0.1:8000/health', timeout=2)
    print('Flask already running, stopping first...')
    # Try to stop gracefully, but if we can't access proc, we'll start a new one anyway
except:
    print('No Flask server detected on port 8000')

# Launch Flask API in background subprocess
print('Launching Flask API from', APP_PATH)
proc = subprocess.Popen(
    [sys.executable, str(APP_PATH)],
    cwd=str(PROJECT_ROOT),
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)
print(f'Started Flask (pid {proc.pid})')

# Wait longer for server to start and check output
time.sleep(5)

# Check if process is still running
if proc.poll() is not None:
    print('Flask process terminated unexpectedly!')
    stdout, stderr = proc.communicate()
    print('Process output:', stdout)
    print('Process errors:', stderr)
else:
    print('Flask process still running')

# Health check with more detailed error info
try:
    r = requests.get('http://127.0.0.1:8000/health', timeout=5)
    print('✓ Health check successful:', r.status_code, r.json())
except requests.exceptions.ConnectionError as e:
    print('✗ Connection refused - Flask server not responding')
    print('Check that Flask started successfully above')
    # Try to get process output for debugging
    if proc.poll() is not None:
        stdout, stderr = proc.communicate()
        print('Flask process output:', stdout)
except Exception as e:
    print('✗ Health check failed with error:', e)


Flask already running, stopping first...
Launching Flask API from /Users/Kushal/FRE/bootcamp_kushal_mamillapalli/project/app.py
Started Flask (pid 6323)
Flask process terminated unexpectedly!
Process output:  * Serving Flask app 'app'
 * Debug mode: on
Address already in use
Port 8000 is in use by another program. Either identify and stop that program, or start the server with a different port.

Process errors: None
✓ Health check successful: 200 {'status': 'ok'}


### Troubleshooting: Run this cell if Flask failed to start


In [5]:
# Troubleshooting: run this cell if Flask server failed to start

# 1. Check if the app.py file exists and is valid
print('Checking app.py...')
print('APP_PATH exists:', APP_PATH.exists())
if APP_PATH.exists():
    with open(APP_PATH, 'r') as f:
        content = f.read()
        print(f'app.py size: {len(content)} characters')
        print('Contains Flask app:', 'Flask(__name__)' in content)

# 2. Try running Flask directly in the terminal (alternative method)
print('\nAlternative: You can manually run Flask in terminal:')
print(f'cd {PROJECT_ROOT}')
print('conda activate fe-course')
print('python app.py')

# 3. Check what's using port 8000
print('\nChecking port 8000...')
import socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
result = sock.connect_ex(('127.0.0.1', 8000))
sock.close()
if result == 0:
    print('Port 8000 is in use')
else:
    print('Port 8000 is available')

# 4. Test model_service functions directly (without Flask)
print('\nTesting model_service functions directly...')
try:
    # Test that model exists and can be loaded
    model_check, feats_check = load_model()
    print('✓ Model loads successfully')
    
    # Test prediction without Flask
    test_row = {c: 0.0 for c in FEATURE_COLUMNS}
    test_row['Amount'] = 100.0
    
    probs = predict_proba(model_check, [test_row], feats_check)
    labels = predict_label(model_check, [test_row], feats_check)
    
    print(f'✓ Direct prediction works: prob={probs[0]:.4f}, label={labels[0]}')
    
except Exception as e:
    print('✗ Model service error:', e)

print('\n=== If Flask still fails ===')
print('1. Make sure you are in the fe-course conda environment')
print('2. Check that Flask is installed: pip list | grep -i flask')
print('3. Try running the app manually in terminal as shown above')
print('4. The model_service functions work directly (see test above)')
print('5. You can proceed with the rest of the notebook using direct function calls')


Checking app.py...
APP_PATH exists: True
app.py size: 4292 characters
Contains Flask app: True

Alternative: You can manually run Flask in terminal:
cd /Users/Kushal/FRE/bootcamp_kushal_mamillapalli/project
conda activate fe-course
python app.py

Checking port 8000...
Port 8000 is in use

Testing model_service functions directly...
✓ Model loads successfully
✓ Direct prediction works: prob=1.0000, label=1

=== If Flask still fails ===
1. Make sure you are in the fe-course conda environment
2. Check that Flask is installed: pip list | grep -i flask
3. Try running the app manually in terminal as shown above
4. The model_service functions work directly (see test above)
5. You can proceed with the rest of the notebook using direct function calls


/Users/Kushal/miniconda3/envs/fe-course/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/Kushal/miniconda3/envs/fe-course/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [8]:
# Test API endpoints (if Flask is running) or use direct functions
print('Testing API endpoints and fallback to direct function calls...\n')

# First check if Flask is responding
flask_running = False
try:
    health_resp = requests.get('http://127.0.0.1:8000/health', timeout=3)
    flask_running = health_resp.status_code == 200
    print(f'✓ Flask API is running: {health_resp.json()}')
except:
    print('✗ Flask API not responding - will use direct function calls')

if flask_running:
    print('\n=== API ENDPOINT TESTING ===')
    
    # 1. POST /predict with JSON payload
    row = {c: 0.0 for c in FEATURE_COLUMNS}
    row['Amount'] = 123.45
    row['Time'] = 10.0
    payload = {'rows': [row], 'threshold': 0.5}
    
    try:
        response = requests.post('http://127.0.0.1:8000/predict', json=payload, timeout=10)
        print('✓ POST /predict with rows:', response.status_code, response.json())
    except Exception as e:
        print('✗ POST /predict failed:', e)
    
    # 2. Features array format
    features_array = [0.0] * len(FEATURE_COLUMNS)
    features_array[-1] = 123.45  # Amount is last in FEATURE_COLUMNS
    payload_features = {'features': [features_array], 'threshold': 0.3}
    
    try:
        response = requests.post('http://127.0.0.1:8000/predict', json=payload_features, timeout=10)
        print('✓ POST /predict with features array:', response.status_code, response.json())
    except Exception as e:
        print('✗ Features array test failed:', e)
    
    # 3. GET path parameters
    try:
        response2 = requests.get('http://127.0.0.1:8000/predict/123.45', timeout=5)
        print('✓ GET /predict/<amount>:', response2.status_code, response2.json())
    except Exception as e:
        print('✗ GET /predict/<amount> failed:', e)
    
    # 4. GET with amount and time
    try:
        response3 = requests.get('http://127.0.0.1:8000/predict/123.45/10.0', timeout=5)
        print('✓ GET /predict/<amount>/<time>:', response3.status_code, response3.json())
    except Exception as e:
        print('✗ GET /predict/<amount>/<time> failed:', e)

else:
    print('\n=== DIRECT FUNCTION TESTING (Flask alternative) ===')
    
    # Test the same functionality using direct model_service calls
    try:
        # Load model once
        model_direct, feats_direct = load_model()
        
        # Test 1: Row-based prediction (like API)
        row = {c: 0.0 for c in FEATURE_COLUMNS}
        row['Amount'] = 123.45
        row['Time'] = 10.0
        
        probs = predict_proba(model_direct, [row], feats_direct)
        labels = predict_label(model_direct, [row], feats_direct, threshold=0.5)
        print(f'✓ Direct prediction (row format): prob={probs[0]:.4f}, label={labels[0]}')
        
        # Test 2: Features array format (like API)
        # Create a proper row from features array like app.py does
        from src.model_service import predict_proba, predict_label
        
        features_array = [0.0] * len(FEATURE_COLUMNS)
        features_array[-1] = 123.45  # Set Amount
        
        # Convert features array to row dict (like _features_list_to_row in app.py)
        row_from_array = {col: float(features_array[i]) for i, col in enumerate(FEATURE_COLUMNS)}
        
        probs2 = predict_proba(model_direct, [row_from_array], feats_direct)
        labels2 = predict_label(model_direct, [row_from_array], feats_direct, threshold=0.3)
        print(f'✓ Direct prediction (features array): prob={probs2[0]:.4f}, label={labels2[0]}')
        
        # Test 3: Path parameter simulation
        amount_only_row = {c: 0.0 for c in FEATURE_COLUMNS}
        amount_only_row['Amount'] = 123.45
        
        probs3 = predict_proba(model_direct, [amount_only_row], feats_direct)
        labels3 = predict_label(model_direct, [amount_only_row], feats_direct, threshold=0.5)
        print(f'✓ Direct prediction (amount only): prob={probs3[0]:.4f}, label={labels3[0]}')
        
        print('\n✓ All model_service functions work correctly!')
        print('The productization is successful - Flask API adds web interface to these functions')
        
    except Exception as e:
        print(f'✗ Direct function testing failed: {e}')


Testing API endpoints and fallback to direct function calls...

✓ Flask API is running: {'status': 'ok'}

=== API ENDPOINT TESTING ===
✓ POST /predict with rows: 200 {'labels': [1], 'probs': [1.0]}
✓ POST /predict with features array: 200 {'labels': [1], 'probs': [1.0]}
✓ GET /predict/<amount>: 200 {'labels': [1], 'probs': [1.0]}
✓ GET /predict/<amount>/<time>: 200 {'labels': [1], 'probs': [1.0]}


## 4. Cleanup and Summary


In [9]:
# Stop the Flask process
try:
    proc.terminate()
    try:
        proc.wait(timeout=5)
    except subprocess.TimeoutExpired:
        proc.kill()
    print('Flask process terminated.')
except Exception as e:
    print('Failed to terminate Flask process:', e)


Flask process terminated.
